In [3]:
!pip install -q git+https://github.com/ultralytics/ultralytics.git
!pip install paho-mqtt==1.6.1


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [18]:
from ultralytics import YOLO
import paho.mqtt.client as mqtt
import time

BROKER = "broker.emqx.io"
PORT = 1883
TOPIC_PUBLISH = "21004173/topic"

def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")

def on_publish(client, userdata, mid):
    print(f"Message Published with mid: {mid}")

def publish_message(predicted_label):
    try:
        client = mqtt.Client()

        client.on_connect = on_connect
        client.on_publish = on_publish

        print("Connecting to broker...")
        client.connect(BROKER, PORT, 60)

        client.loop_start()

        time.sleep(1)

        result = client.publish(TOPIC_PUBLISH, predicted_label, qos=1)
        result.wait_for_publish()
        print(f"Sent message: {predicted_label} to topic: {TOPIC_PUBLISH}")

        time.sleep(1)
        client.loop_stop()
        client.disconnect()

    except Exception as e:
        print(f"Error in publishing message: {e}")

model = YOLO("/content/drive/MyDrive/21004173/21004170 (1).pt")

image_path = "/content/drive/MyDrive/21004173/000000_jpg.rf.9c2d93c681b7d38d50bf4f096a6baf03.jpg"

try:
    results = model.predict(source=image_path, conf=0.25, save=True)

    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy())
            class_name = model.names[class_id]
            print(f"Class detected: {class_name}")
            publish_message(class_name)
            time.sleep(1)

except Exception as e:
    print(f"Error in detection: {e}")


image 1/1 /content/drive/MyDrive/21004173/000000_jpg.rf.9c2d93c681b7d38d50bf4f096a6baf03.jpg: 640x640 1 Speed Limit 30, 326.1ms
Speed: 8.1ms preprocess, 326.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict14
Class detected: Speed Limit 30
Connecting to broker...
Connected with result code 0


<ipython-input-18-bc7616fc0305>:64: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  class_id = int(box.cls.cpu().numpy())


Message Published with mid: 1
Sent message: Speed Limit 30 to topic: 21004173/topic
